# Import

In [1]:
from airbnb_mysql import *
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Connexion to database

In [2]:
port_name = '3306'
user_name = 'root'
user_password = 'root'
host_name = 'localhost'
db = 'airbnb'

connection = create_db_connection(user=user_name, password=user_name, host=host_name, db_name=db, port=port_name)

query = """
        SELECT * FROM apartment;
"""

col_name = ["id", "neighbour", "lat", "long", "r_type", "price", "nbr_rev", "rev_per_month"]
data = db_to_df(connection, query, col_name)

MySQL Database connection successful


# Preprocessing

In [3]:
df = data.drop(["id","nbr_rev","rev_per_month"],axis = 1)

In [4]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import RobustScaler

In [5]:
def encoding(df_col):
    X = df_col
    encoder = LabelBinarizer()
    X_t = encoder.fit_transform(X)
    column = encoder.classes_
    return X_t, column

In [6]:
def scaling(df_col):
    X = df_col
    X = X.to_numpy()
    X = np.reshape(X,(-1,1))
    scaler = RobustScaler().fit(X)
    X_t = scaler.transform(X)
    return X_t

In [7]:
encod_room, room_col = encoding(df['r_type'])
room = pd.DataFrame(encod_room,columns=room_col)


encod_neigh, neigh_col = encoding(df['neighbour'])
neigh = pd.DataFrame(encod_neigh,columns=neigh_col)
scaled_price = scaling(df['price'])

In [8]:
frames = [df,room,neigh]
clean_df = pd.concat(frames,axis=1)
clean_df = clean_df.drop(["neighbour","r_type"],axis=1)

clean_df['scaled_price']= scaled_price
clean_df = clean_df.drop(['price'],axis=1)

clean_df = clean_df.drop(['lat','long'],axis =1)

# Train test split

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
train_set,test_set = train_test_split(clean_df,test_size=0.3,random_state=0)

In [11]:
train_set

,Entire home/apt,Hotel room,Private room,Shared room,Batignolles-Monceau,Bourse,Buttes-Chaumont,Buttes-Montmartre,EntrepÃ´t,Gobelins,...,OpÃ©ra,Palais-Bourbon,PanthÃ©on,Passy,Popincourt,Reuilly,Temple,Vaugirard,Ãx89lysÃ©e,scaled_price
47953,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0.283333
26585,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,-0.500000
61076,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-0.333333
27675,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.050000
43888,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0.550000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41993,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0.750000
21243,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-0.650000
45891,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,-0.333333
42613,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.666667


# Regression

In [12]:
from sklearn.linear_model import SGDRegressor

In [13]:
y_train = train_set['scaled_price']
X_train = train_set.drop('scaled_price',axis=1)
reg = SGDRegressor(max_iter=1000, tol=1e-3)
reg.fit(X_train,y_train)

SGDRegressor()

In [14]:
y_test = test_set['scaled_price']
X_test = test_set.drop('scaled_price',axis=1)
y_test_predicted = reg.predict(X_test)

# Evaluation

In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import validation_curve
from sklearn.metrics import r2_score

In [16]:
r2_score()

0.02978042555332583
